# 🏗️ Week 11-12 · Notebook 07 · Infrastructure as Code (IaC) with Terraform

This notebook introduces Infrastructure as Code (IaC) using Terraform to define, provision, and manage the entire cloud infrastructure for our Manufacturing Copilot. This ensures our deployments are reproducible, auditable, and consistent across all environments.


## 🎯 Learning Objectives

- **Codify Cloud Infrastructure:** Use Terraform to define all the necessary GCP resources for the copilot, including Cloud Run services, VPC connectors, Secret Manager secrets, and logging configurations.
- **Create Reusable Modules:** Structure the Terraform code into reusable modules (e.g., a `cloud_run` module) to reduce duplication and simplify management.
- **Manage Multiple Environments:** Use Terraform workspaces and variable files (`.tfvars`) to manage differences between `staging` and `production` environments with minimal configuration drift.
- **Implement a Safe Workflow:** Understand and use the `terraform plan` and `terraform apply` workflow to safely preview and apply changes to the infrastructure.
- **Integrate Policy as Code:** Introduce the concept of using tools like Open Policy Agent (OPA) to enforce organizational policies (e.g., "all storage buckets must have versioning enabled") before infrastructure is deployed.


## 🧩 Scenario: From Manual Clicks to Automated Infrastructure

The Manufacturing Copilot platform is growing. Multiple teams are now contributing, and manually creating resources in the GCP console ("click-ops") is becoming slow, error-prone, and impossible to audit. A recent incident, where a staging firewall rule was accidentally applied to production, has made it clear that a new approach is needed.

**The Mandate:** All cloud infrastructure must be managed through code. The state of the infrastructure must be stored remotely in a GCS bucket with state locking enabled to prevent concurrent, conflicting changes.


In [ ]:
# --- terraform/main.tf ---
# This Python cell creates the Terraform configuration files in a `terraform` directory.

from pathlib import Path
from textwrap import dedent

# Create the directory structure
terraform_dir = Path("terraform_example")
modules_dir = terraform_dir / "modules"
(modules_dir / "cloud_run").mkdir(parents=True, exist_ok=True)
(modules_dir / "secrets").mkdir(parents=True, exist_ok=True)

# --- Root main.tf ---
main_tf_content = dedent("""
    terraform {
      required_version = ">= 1.5.0"

      # Configure the backend to store state in a GCS bucket
      # This is critical for team collaboration and state locking.
      backend "gcs" {
        bucket = "manufacturing-copilot-tf-state"
        prefix = "prod" # Use 'staging' or 'prod' depending on the workspace
      }
    }

    provider "google" {
      project = var.project_id
      region  = var.region
    }

    # --- Resources ---

    # Call the reusable module to create our Cloud Run service
    module "copilot_service" {
      source = "./modules/cloud_run"

      # Pass variables to the module
      service_name  = "manufacturing-copilot-api"
      project_id    = var.project_id
      region        = var.region
      image_uri     = var.image_uri
      max_instances = var.max_instances
      min_instances = var.min_instances
      
      # Pass the secret configurations to the module
      secrets = {
        "ENV_OPENAI_API_KEY" = module.secrets.openai_api_key.secret_id,
        "ENV_DB_PASSWORD"    = module.secrets.db_password.secret_id
      }
    }

    # Call the module to manage our secrets
    module "secrets" {
      source = "./modules/secrets"

      project_id = var.project_id
      secrets = {
        "openai-api-key" = "The API key for OpenAI services.",
        "db-password"    = "The password for the production database."
      }
    }
""")

# --- Root variables.tf ---
variables_tf_content = dedent("""
    variable "project_id" {
      description = "The GCP project ID to deploy resources into."
      type        = string
    }

    variable "region" {
      description = "The GCP region for the deployment."
      type        = string
      default     = "us-central1"
    }

    variable "image_uri" {
      description = "The full URI of the Docker image to deploy."
      type        = string
    }

    variable "max_instances" {
      description = "The maximum number of instances for the Cloud Run service."
      type        = number
      default     = 4
    }
    
    variable "min_instances" {
      description = "The minimum number of instances for the Cloud Run service."
      type        = number
      default     = 0
    }
""")

# Write the files
(terraform_dir / "main.tf").write_text(main_tf_content)
(terraform_dir / "variables.tf").write_text(variables_tf_content)

print("--- terraform/main.tf ---")
print(main_tf_content)
print("\n--- terraform/variables.tf ---")
print(variables_tf_content)


# --- terraform/modules/cloud_run/main.tf ---
cloud_run_module_main = dedent("""
    # This module defines a reusable Google Cloud Run service.

    resource "google_cloud_run_v2_service" "default" {
      name     = var.service_name
      location = var.region
      project  = var.project_id

      template {
        containers {
          image = var.image_uri
          
          # Dynamically create environment variables from the secrets map
          dynamic "env" {
            for_each = var.secrets
            content {
              name = env.key
              value_from {
                secret_key_ref {
                  secret  = split("/", env.value)[3] # Extract secret name from full ID
                  version = "latest"
                }
              }
            }
          }
        }

        scaling {
          min_instance_count = var.min_instances
          max_instance_count = var.max_instances
        }
      }
    }
""")

# --- terraform/modules/cloud_run/variables.tf ---
cloud_run_module_variables = dedent("""
    variable "service_name" { type = string }
    variable "project_id" { type = string }
    variable "region" { type = string }
    variable "image_uri" { type = string }
    variable "min_instances" { type = number }
    variable "max_instances" { type = number }
    variable "secrets" {
      description = "A map of environment variable names to secret IDs."
      type        = map(string)
      default     = {}
    }
""")

# Write the module files
(modules_dir / "cloud_run" / "main.tf").write_text(cloud_run_module_main)
(modules_dir / "cloud_run" / "variables.tf").write_text(cloud_run_module_variables)

print("--- terraform/modules/cloud_run/main.tf ---")
print(cloud_run_module_main)
